<a href="https://colab.research.google.com/github/czeacach/fashion_designers/blob/main/wdt_nationality_production.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Production des nationalités de l'effectif renseigné dans Wikidata
Dans ce carnet est proposée la méthode de production des nationalités de la population Wikidata retenue.

In [ ]:
!pip install sparqlwrapper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.9/531.9 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 3.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
from SPARQLWrapper import SPARQLWrapper, SPARQLWrapper2, JSON, TURTLE, XML, RDFXML

In [ ]:
!pip install itables

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.7/221.7 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 16.9 MB/s eta 0:00:00


In [ ]:
### https://mwouts.github.io/itables/quick_start.html

from itables import init_notebook_mode, show

init_notebook_mode(all_interactive=False)

In [ ]:
### Librairies déjà installées avec Python
import pprint
import csv
import sys

import sqlite3 as sql

import time
import datetime
from dateutil import parser

from importlib import reload
from shutil import copyfile

In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/Colab_notebooks')
import sparql_functions as spqf

#Préparer les données

## SPARQL Query qui récupère les données
On exécute la requête grâce à une fonction de la librairie locale qui réalise la mise en forme

In [ ]:
## define SPARQL enpoint
endpoint = "https://query.wikidata.org/sparql"

In [ ]:
query = """
SELECT DISTINCT ?item ?country ?countryLabel
WHERE {
    {?item wdt:P106 wd:Q3501317} # occupation is fashion designer.
        UNION
        {?item wdt:P101 wd:Q29583} # field of work is fashion design
?item wdt:P31 wd:Q5; # Any instance of a human.
        wdt:P569 ?birthDate.
 ?item wdt:P27 ?country
    BIND(REPLACE(str(?birthDate), "(.*)([0-9]{4})(.*)", "$2") AS ?year)
    FILTER(xsd:integer(?year) > 1800 && xsd:integer(?year) < 2001)
SERVICE wikibase:label { bd:serviceParam wikibase:language "en" }
        }
     ORDER BY ?year
     """

In [ ]:
### Executer la requête avec les fonctions de la librairie locale
qr = spqf.get_json_sparql_result(endpoint,query)

In [ ]:
r = [l for l in spqf.sparql_result_to_list(qr)]
print(len(r))
r[:3]

3173


[['http://www.wikidata.org/entity/Q3524312',
  'http://www.wikidata.org/entity/Q142',
  'France'],
 ['http://www.wikidata.org/entity/Q62128099',
  'http://www.wikidata.org/entity/Q62633',
  'Grand Principality of Finland'],
 ['http://www.wikidata.org/entity/Q4721481',
  'http://www.wikidata.org/entity/Q142',
  'France']]

In [ ]:
### Créer un DataFrame à partir du résultat
df_r = pd.DataFrame(r)
df_r.columns = ['personUri', 'nationalityUri', 'nationalityLabel']
df_r.head()

,personUri,nationalityUri,nationalityLabel
0,http://www.wikidata.org/entity/Q3524312,http://www.wikidata.org/entity/Q142,France
1,http://www.wikidata.org/entity/Q62128099,http://www.wikidata.org/entity/Q62633,Grand Principality of Finland
2,http://www.wikidata.org/entity/Q4721481,http://www.wikidata.org/entity/Q142,France
3,http://www.wikidata.org/entity/Q4962968,http://www.wikidata.org/entity/Q34,Sweden
4,http://www.wikidata.org/entity/Q5363058,http://www.wikidata.org/entity/Q30,United States of America


In [ ]:
df_r.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3173 entries, 0 to 3172
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   personUri         3173 non-null   object
 1   nationalityUri    3173 non-null   object
 2   nationalityLabel  3173 non-null   object
dtypes: object(3)
memory usage: 74.5+ KB


## Créer une nouvelle table dans la base de données SQLite
On va stocker dans cette table le résultat de la requête SPARQL préalablement transformé en DataFrame

In [ ]:
### Se connecter à la base de données dans laquelle on va insérer
# le résultat de la requête SPARQL
#chemin d'acces de la base de données directement connectée à l'ordinateur via Google Drive Desktop
'/content/drive/MyDrive/dossier de travail/fashion_designers_data_analysis.db'
cn = sql.connect('/content/drive/MyDrive/dossier de travail/fashion_designers_data_analysis.db')
cn

In [ ]:
### Créer une nouvelle table contenant le DataFrame
# Si on tente de la recréer, alors qu'elle existe déjà,
# un message d'erreur est renvoyé
try:
    l = df_r.to_sql(name='wdt_person_nationality', con=cn, if_exists='fail')
except Exception as e:
    print('Erreur: ',  e)

Erreur:  Table 'wdt_person_nationality' already exists.


In [ ]:
### Vérifier que les données ont été importées correctement
cur = cn.cursor()
l = cur.execute("SELECT * FROM wdt_person_nationality limit 3").fetchall()
### On a mis le résultat de la requête SQL
# dans une liste 'l' qu'on affiche avec une boucle 'for'
# dans le cadre d'une 'list comprehension'
a = [print(e) for e in l]

(0, 'http://www.wikidata.org/entity/Q3524312', 'http://www.wikidata.org/entity/Q142', 'France')
(1, 'http://www.wikidata.org/entity/Q62128099', 'http://www.wikidata.org/entity/Q62633', 'Grand Principality of Finland')
(2, 'http://www.wikidata.org/entity/Q4721481', 'http://www.wikidata.org/entity/Q142', 'France')


In [ ]:
### Vérifier que les données ont été importées correctement
cur = cn.cursor()
l = cur.execute("SELECT COUNT(*) FROM wdt_person_nationality ").fetchone()
print('Nombre de lignes de la table:', l[0])

Nombre de lignes de la table: 3165


#Créer une table qui contient les nationalités

In [ ]:
q="""
-- noter que la fonction TRIM élimine les éventuels espaces en début ou fin de chaine de charactères
-- la fonction LOWER met tout au minuscule
SELECT TRIM(nationalityUri) as nationalityUri, LOWER(TRIM(nationalityLabel)) AS nationalityLabel, COUNT(*) as effectif
FROM wdt_person_nationality
GROUP BY TRIM(nationalityUri), LOWER(TRIM(nationalityLabel))
ORDER BY effectif DESC
"""
cur = cn.cursor()
r = cur.execute(q).fetchall()
rdf = pd.DataFrame(r,columns=['uri', 'label', 'freq'])
rdf.head()

,uri,label,freq
0,http://www.wikidata.org/entity/Q30,united states of america,556
1,http://www.wikidata.org/entity/Q145,united kingdom,235
2,http://www.wikidata.org/entity/Q142,france,213
3,http://www.wikidata.org/entity/Q17,japan,163
4,http://www.wikidata.org/entity/Q38,italy,150


In [ ]:
rdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 164 entries, 0 to 163
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   uri     164 non-null    object
 1   label   164 non-null    object
 2   freq    164 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 4.0+ KB


In [ ]:
#Creer une table Sqlite avec les colonnes dont on a besoin : index /nationalityUri / nationalityLabel

In [ ]:
### Après avoir crée avec SQLite Studio une table 'wdt_nationality'
# la remplir avec une ligne par métier


q2 = """
INSERT INTO wdt_nationality (nationlityUri,nationalityLabel)
SELECT DISTINCT TRIM(nationalityUri), LOWER(TRIM(nationalityLabel))
FROM wdt_person_nationality;
"""

## Attention : la requête est commentée pour éviter
# de la réexécuter par mégarde: en effet, cette table
# servira au codage des domaines — ne pas l'effacer

cur = cn.cursor()
#r = cur.execute(q2)
#cn.commit()

In [ ]:
# la fonction est bien effectuée: la table est remplie avec les uri et les label dans les colonnes respectives

## Vérifier l'importation

In [ ]:
### Le nombre de nationalités différentes: noter qu'il y des répétitions
## et inconsistances dans les données de Wikidata

q="""
SELECT COUNT(*)
FROM wdt_nationality
"""
cur = cn.cursor()
r = cur.execute(q).fetchone()
print('Nombre de nationalités différentes:' , r[0])

Nombre de nationalités différentes: 164


In [ ]:
## Vérifier que le DataFrame rdf contient le même nombre de modalités
print(len(rdf))

164


In [ ]:
### Quelques nationalités
# pour mieux explorer parcourir la table dans DBeaver

q="""
SELECT *
FROM wdt_nationality
LIMIT 5
"""
cur = cn.cursor()
r = cur.execute(q).fetchall()
pprint.pprint(r)

[(1, None, 'http://www.wikidata.org/entity/Q142', 'france', 1, 3),
 (2,
  None,
  'http://www.wikidata.org/entity/Q62633',
  'grand principality of finland',
  1,
  2),
 (3, None, 'http://www.wikidata.org/entity/Q34', 'sweden', 1, 2),
 (4,
  None,
  'http://www.wikidata.org/entity/Q30',
  'united states of america',
  4,
  None),
 (5,
  None,
  'http://www.wikidata.org/entity/Q174193',
  'united kingdom of great britain and ireland',
  1,
  2)]


# Vérifier les doublons dans la table wdt_personne

In [ ]:
# requête pour chercher les doublons dans la table personne
q="""
SELECT personUri, personLabel, COUNT(*) AS occurrences
FROM wdt_personne
GROUP BY personUri
HAVING COUNT(*) > 1;
"""
cur = cn.cursor()
r = cur.execute(q).fetchall()
pprint.pprint(r)

[('http://www.wikidata.org/entity/Q106151771', 'Madeleine Laferrière', 2),
 ('http://www.wikidata.org/entity/Q106318387', 'Betty Brader-Ashley', 2),
 ('http://www.wikidata.org/entity/Q107328394', 'Jenny Meirens', 2),
 ('http://www.wikidata.org/entity/Q108880375', 'Christian Roth', 2),
 ('http://www.wikidata.org/entity/Q110344105', 'Mark Eisen', 2),
 ('http://www.wikidata.org/entity/Q111577', 'Howard Greer', 2),
 ('http://www.wikidata.org/entity/Q145245', 'Irene Lentz', 2),
 ('http://www.wikidata.org/entity/Q15956259', 'Sara Forsberg', 2),
 ('http://www.wikidata.org/entity/Q16221890', 'Hannah Marshall', 2),
 ('http://www.wikidata.org/entity/Q18209916', 'Olivier Rousteing', 2),
 ('http://www.wikidata.org/entity/Q20205233', 'Coco Johnsen', 2),
 ('http://www.wikidata.org/entity/Q2168925', 'Armand Basi', 2),
 ('http://www.wikidata.org/entity/Q242342', 'Miuccia Prada', 2),
 ('http://www.wikidata.org/entity/Q33215154', 'Rose Torrente-Mett', 2),
 ('http://www.wikidata.org/entity/Q3470884', 'Sa

In [ ]:
#requête pour supprimer les doublons de la table wdt_personne

q="""
DELETE FROM wdt_personne
WHERE rowid NOT IN (
    SELECT MIN(rowid)
    FROM wdt_personne
    GROUP BY personUri
);
"""
cur = cn.cursor()
r = cur.execute(q).fetchall()
pprint.pprint(r)

[]


In [ ]:
#requête pour vérifier que les doublons ont bien été supprimés
q="""
SELECT personUri, personLabel, COUNT(*) AS occurrences
FROM wdt_personne
GROUP BY personUri
HAVING COUNT(*) > 1;
"""
cur = cn.cursor()
r = cur.execute(q).fetchall()
pprint.pprint(r)

[]


#Vérifier les doublons dans la table wdt_person_nationality

In [ ]:
# requête pour chercher les doublons dans la table wdt_person_nationality
# pour voir combien de personnes ont plus d'une nationalités
q="""
SELECT personUri, COUNT(*) AS occurrences
FROM wdt_person_nationality
GROUP BY personUri
HAVING COUNT(*) > 1;
"""
cur = cn.cursor()
r = cur.execute(q).fetchall()
pprint.pprint(r)

[('http://www.wikidata.org/entity/Q1051455', 2),
 ('http://www.wikidata.org/entity/Q10535842', 2),
 ('http://www.wikidata.org/entity/Q105672918', 2),
 ('http://www.wikidata.org/entity/Q106196010', 2),
 ('http://www.wikidata.org/entity/Q106772192', 2),
 ('http://www.wikidata.org/entity/Q107277233', 2),
 ('http://www.wikidata.org/entity/Q10857618', 2),
 ('http://www.wikidata.org/entity/Q10872855', 2),
 ('http://www.wikidata.org/entity/Q108880375', 2),
 ('http://www.wikidata.org/entity/Q108920008', 2),
 ('http://www.wikidata.org/entity/Q109341687', 5),
 ('http://www.wikidata.org/entity/Q110604850', 2),
 ('http://www.wikidata.org/entity/Q110670210', 2),
 ('http://www.wikidata.org/entity/Q110795365', 2),
 ('http://www.wikidata.org/entity/Q110965126', 2),
 ('http://www.wikidata.org/entity/Q111026013', 2),
 ('http://www.wikidata.org/entity/Q111578323', 2),
 ('http://www.wikidata.org/entity/Q111579862', 2),
 ('http://www.wikidata.org/entity/Q111588744', 2),
 ('http://www.wikidata.org/entity/Q1

In [ ]:
# requête pour compter les doublons dans la table wdt_person_nationality
# pour voir combien de personnes ont plus d'une nationalités
# il y a 259 instances qui ont plusieurs nationalités dans la table wdt_person_nationality
q="""
SELECT COUNT(*) AS total_doublons
FROM (
    SELECT personUri
    FROM wdt_person_nationality
    GROUP BY personUri
    HAVING COUNT(*) > 1
) AS doublons
"""
cur = cn.cursor()
r = cur.execute(q).fetchall()
pprint.pprint(r)

[(259,)]


In [ ]:
# requête qui compte le nombre d'occurence pour chaque personne
q="""
SELECT occurrences, COUNT(*) AS count_personnes
FROM (
    SELECT personUri, COUNT(*) AS occurrences
    FROM wdt_person_nationality
    GROUP BY personUri
) AS occurences_par_personne
GROUP BY occurrences
ORDER BY occurrences
"""
cur = cn.cursor()
r = cur.execute(q).fetchall()
pprint.pprint(r)

[(1, 2613), (2, 233), (3, 21), (4, 2), (5, 3)]


In [ ]:
### Créer un DataFrame à partir du résultat
df_r = pd.DataFrame(r)
df_r.columns = ['occurences', 'count_occurences']
df_r.head()


,occurences,count_occurences
0,1,2613
1,2,233
2,3,21
3,4,2
4,5,3
